In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in Audio Features

In [2]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

#### Drops all uncommon songs

In [3]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')

In [4]:
popularity_ratings = pd.read_pickle('../Pickle_Files/Song_Popularity.pkl')

In [5]:
Compare = pd.merge(Compare,popularity_ratings[['Popularity_Rating_Normalized','track_uri']],on='track_uri')

#### Ordering tracks

In [6]:
Compare = Compare.sort_values('track_uri',ascending=True)

In [7]:
Compare = Compare.reset_index().drop(['index'],axis=1)

In [8]:
len(Compare)

2254670

#### Creating Dataframe 'Cosine'

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
cosine = pd.read_pickle('../Pickle_Files/cosine_dataframe.pkl')

In [11]:
X = cosine.values

#### Normalizing values before cosine calculated

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit(cosine.values).transform(cosine.values)

# Playlist Continuation System
<br>-  This Playlist Continuation System continues a set playlist for the length of N songs

#### This value for N sets the number of songs you would like to use to continue the playlist

In [13]:
N = 100

#### This chooses a random playlist to continue

In [14]:
mylist = track_elems.pid.unique()

import random

sample_size = 1
sorted_sample = [mylist[i] for i in sorted(random.sample(range(len(mylist)), sample_size))]

df = {name: track_elems.loc[track_elems['pid'] == name] for name in sorted_sample}

#### Gives correct index value to each song in random playlist

In [15]:
random_playlist = df[sorted_sample[0]]
random_playlist = pd.merge(Compare.reset_index(),random_playlist[['track_uri']],on='track_uri').set_index('index')

In [16]:
random_playlist

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,Popularity_Rating_Normalized
index,,,,,,,,,
195375,spotify:track:0fioLzGM8ngbD1w6fMmm45,In The Lonely Hour,spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK,Sam Smith,spotify:artist:2wY79sveU1sp5g7SokKOiI,239316,None,I'm Not The Only One,0.853490
213730,spotify:track:0jfsWEn4tP9oWzsLSNk3Jq,The Gift,spotify:album:3sHGkl7UfE9JGDLv5sC7H5,Pia Mia,spotify:artist:1BhWF9W2PngtPSyobKg0rP,205786,None,Red Love,0.432288
232870,spotify:track:0nnRUsgfm4uQUYTCIH9NZy,The Pinkprint,spotify:album:40XGTQ7FN6Y3dZXJhKBe96,Nicki Minaj,spotify:artist:0hCNtLu0JehylgoiP8L4Gh,268000,None,Pills N Potions,0.746374
299352,spotify:track:11pfbduq7fbQxmXH03OKo7,In The Lonely Hour,spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK,Sam Smith,spotify:artist:2wY79sveU1sp5g7SokKOiI,172452,None,Not In That Way,0.702785
360578,spotify:track:1EtU16em39vcCVMx2mGpHq,Reflection,spotify:album:6IBUil5AQA6wPLCzpYrlMI,Fifth Harmony,spotify:artist:1l8Fu6IkuTP0U5QetQJ5Xt,171413,None,BO$$,0.752999
414100,spotify:track:1QJLm4mdmqRkXXbHc4pYWY,The New Classic,spotify:album:6lBTYoX7ZzU2Xbf8RHAv0u,Iggy Azalea,spotify:artist:5yG7ZAZafVaAlMTeBybKAL,220748,None,Change Your Life,0.626937
445250,spotify:track:1X2Zd5wKGbY1oKzb8dzJRy,My Everything,spotify:album:2ZnzBwKw4e2SHpGvOTWnj4,Ariana Grande,spotify:artist:66CXWjxzNUsdJxJ2JdwvnR,214840,None,Break Free,0.799838
446758,spotify:track:1XMjzq2p5v4CSWf8v6Diuh,The Gift,spotify:album:3sHGkl7UfE9JGDLv5sC7H5,Pia Mia,spotify:artist:1BhWF9W2PngtPSyobKg0rP,184813,None,On My Mind,0.427611
576861,spotify:track:1zCi4cVFqe6ja16MeGZKRN,"good kid, m.A.A.d city",spotify:album:6PBZN8cbwkqm1ERj2BGXJ1,Kendrick Lamar,spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg,300160,None,Poetic Justice,0.831996


#### Playlist Continuation System

In [17]:
def Playlist_Continuation_System(df):
    
    dfvalues = df.index.values
    
    num_tracks_desired = 8 ## Recommends eight songs, for large values of N this value should be increased
    def rec_top_n(n):
        return np.argsort(cosine_similarity(X, X[dfvalues]), axis=0)[::-1][1:n] # Cosine similarity finds closest to each song
    # Prints index of all recommended songs
    print(rec_top_n(num_tracks_desired+1))
    playlist_continuation = rec_top_n(num_tracks_desired+1).flatten()

    playlist_continuation = Compare.iloc[playlist_continuation]
    playlist_continuation = playlist_continuation.sample(N)
    return playlist_continuation

#### This runs the playlist continuation with the random playlist selected above and prints the next N songs

In [18]:
Playlist_Continuation_System(random_playlist)

[[ 505929  515045  300670  703187  630411 2231176  400388 1638964  215025
  1157912  197180 1630848 1920602 1920602  969563 1545253 1330949  163158
  2093729  623369 1966110 1445239  467680 1000931  725296   95424 1281669
   363319 1684886 1293027  839920  184772  235662 2028456 1999893 1508273
   118224  389338  799740 1808431 2166986 1164175]
 [1903049  991372 1472671 2178187 1814046   18399 2215610 1996257  274631
   277548 1564424 2036861  932133  932133 1986599  874767  671189 1497012
  1400340 1408267  652402 1091888  553093 1284115 1533084  236968 1650209
   755893  515218 1518177   36385 1537419 1435774 2087541  533050 2130514
  1628052 2202843 1473255 1910349 1572590 1114771]
 [2155948 1981025  149220 1314309 1295002  942483  152858  293685   36833
   446469 1737386 1479046  963312  963312 1871118  145080 1721128 1781000
  1947058  381408 1970041  677476   24518 1516535 1994929  881784 1978203
   725553  108216 1147212  352382 2069618 1501551  286273  964510  323330
   764145 

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,Popularity_Rating_Normalized
319722,spotify:track:16Ah4QqH4mgYVXqfC4mdSd,I Told You,spotify:album:5tBOCi2TekXZ1IbrmlBZO4,Tory Lanez,spotify:artist:2jku7tDXc6XoB6MO2hFuqg,228640,None,LUV,0.800316
533050,spotify:track:1poBO3pO6fkqPYG6n8J5cX,This Is How We Do It,spotify:album:6VTrm81s50qmidaYI6EjiQ,Montell Jordan,spotify:artist:0iVrCROxeyon7MZUW3MfzT,277293,None,This Is How We Do It,0.562277
149220,spotify:track:0VqMBJyeeRA4I6mSPO5uqZ,If We're Honest (Deluxe Version),spotify:album:5ZEGGjJdlQdDNhLKYucTJw,Francesca Battistelli,spotify:artist:29ywwKkxfoH7iWwNY1UezA,302577,None,Holy Spirit,0.699915
797382,spotify:track:2kIq7Oq6YNfdUuTrmPC4w1,Hannah Montana Forever,spotify:album:6FUFS9PRGUTyVrsltg1xji,Hannah Montana,spotify:artist:7nU4hB040gTmHm45YYMvqc,177506,None,Ordinary Girl,0.672677
1573499,spotify:track:5QqyRUZeBE04yJxsD1OC0I,Lust For Life,spotify:album:7xYiTrbTL57QO0bb4hXIKo,Lana Del Rey,spotify:artist:00FQb4jTyendYWaN8pK0wa,260672,None,Summer Bummer (feat. A$AP Rocky & Playboi Carti),0.718910
1473255,spotify:track:55FgCRIw5sUucVrt3iXVmP,Catch Up,spotify:album:6kjlTy2BDXFolXLyb8Ryhv,Omar Kadir,spotify:artist:1dZdd4uleOGer6HoYZq62D,188387,None,Catch Up,0.504058
1844139,spotify:track:6MtWTKkTqk2Tg2IdTSUdAW,Plata O Plomo,spotify:album:1hyd7RYmR54Elg3MhZzXJW,Fat Joe,spotify:artist:3ScY9CQxNLQei8Umvpx5g6,226178,None,Cookin,0.513022
1342425,spotify:track:4dAUGcD3zfZhJPyLXFSAUW,Sniffing Vicodin In Paris (Danny Olson Remix) ...,spotify:album:2LMZ8uw5tTl0z9mZns9n8b,blackbear,spotify:artist:2cFrymmkijnjDg9SS92EPM,192428,None,Sniffing Vicodin In Paris (Danny Olson Remix) ...,0.589263
1292646,spotify:track:4STaHtfguDMkWbPEo6Fwcx,PARTYNEXTDOOR 3 (P3),spotify:album:2FXGUAESmG5l9YPrzWPvHI,PARTYNEXTDOOR,spotify:artist:2HPaUgqeutzr3jx5a9WyDV,202661,None,Not Nice,0.771311
212447,spotify:track:0jOfl1nZf6sVBTG08Q4brn,Culture,spotify:album:2AvupjUeMnSffKEV05x222,Migos,spotify:artist:6oMuImdp5ZcFhWP0ESe6mG,232705,None,Call Casting,0.712774


# Conclusion
<br>-After a quick look at the returned continued playlist it can be said that it is good.
<br>-Artists that occur in the random playlist also occur multiple times in the continuation ie: Kendrick Lamar and The Weeknd
<br>-The recommender system covers all the genres included in the random playlist
    - Rap
            -Orginal playlist: Drake,Kendrick Lamar & Iggy Azalea
            -Continuation system: Kendrick Lamar, Travis Scott, Migos & Lil Wayne
    - Pop
            -Orginal playlist: Ed Sheeran, The Weeknd & Ariana Grande
            -Contiinuation system: The Weeknd, Justin Bieber, Katy Perry & Lana Del Rey
<br>**Hence it can be argued that the playlist continuation system is good.**
    